# 🤖 YouTube 자막 추출 + Gemini 요약 + Notion 연동 (통합)

**YouTube 영상 자막을 추출하고, Gemini AI로 요약한 후, Notion에 자동으로 저장하는 완전 자동화 시스템**

### 🎯 통합된 기능:
- ✅ YouTube 자막 추출 (최신 API 사용)
- ✅ Google Gemini Pro AI 요약
- ✅ Notion 페이지 자동 생성 및 저장
- ✅ 사용자 정의 요약 스타일 지원

In [ ]:
# 패키지 import
import asyncio
import json
import logging
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Union
from urllib.parse import urlparse
import re

# 웹 크롤링 및 HTTP
import requests
from bs4 import BeautifulSoup
import validators

# Google GenAI
import google.generativeai as genai
from google.genai import types

# Notion API
from notion_client import Client
import numpy as np
import pandas as pd

# 유튜브 (최신 API)
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound

# 환경변수
from dotenv import load_dotenv
load_dotenv()

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ 모든 패키지가 성공적으로 로드되었습니다!")

## 🔑 API 키 및 환경 설정(설정완료로 패스)

In [ ]:
# 환경 변수 설정
NOTION_TOKEN = os.getenv("NOTION_TOKEN")
PARENT_PAGE_ID = os.getenv("NOTION_PARENT_PAGE") # 상위 페이지 ID
NOTION_DATABASE_ID = os.getenv("NOTION_DATABASE_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  # Gemini API 키

print("🔑 환경 변수 상태:")
print(f"  • NOTION_TOKEN: {'✅ 설정됨' if NOTION_TOKEN else '❌ 설정 필요'}")
print(f"  • PARENT_PAGE_ID: {'✅ 설정됨' if PARENT_PAGE_ID else '❌ 설정 필요'}")
print(f"  • NOTION_DATABASE_ID: {'✅ 설정됨' if NOTION_DATABASE_ID else '❌ 설정 필요'}")
print(f"  • GEMINI_API_KEY: {'✅ 설정됨' if GEMINI_API_KEY else '❌ 설정 필요'}")

if not all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    print("⚠️ 필요한 환경 변수가 설정되지 않았습니다!")
    print("💡 .env 파일에 다음 항목들을 설정해주세요:")
    print("   - NOTION_TOKEN=your_notion_token")
    print("   - notion_parent_page=your_parent_page_id")
    print("   - GOOGLE_AI_API_KEY=your_google_ai_key")

## 🤖 통합 YouTube-Gemini-Notion Agent 클래스

In [ ]:
class YouTubeGeminiNotionAgent:
    """YouTube 자막 추출 + Gemini 요약 + Notion 저장 통합 클래스"""
    
    def __init__(self, notion_token: str, parent_page_id: str, google_ai_api_key: str, database_id: str = None):
        """
        초기화
        
        Args:
            notion_token: Notion API 토큰
            parent_page_id: Notion 상위 페이지 ID
            google_ai_api_key: Google AI API 키
            database_id: Notion 데이터베이스 ID (선택사항)
        """
        # Notion 설정
        self.notion_token = notion_token
        self.parent_page_id = parent_page_id
        self.database_id = database_id
        
        # Gemini 설정
        genai.configure(api_key=google_ai_api_key)
        # self.model = genai.GenerativeModel('gemini-pro')
        self.model = genai.GenerativeModel('gemini-2.5-flash-lite') # 기존연습하던대로 변경
        
        # Notion Client (optional)
        if database_id:
            self.notion_client = Client(auth=notion_token)
        
        print("🚀 YouTube-Gemini-Notion Agent 초기화 완료!")
    
    def extract_youtube_video_id(self, url: str) -> str:
        """유튜브 URL에서 비디오 ID 추출"""
        patterns = [
            r'(?:youtube\.com\/watch\?v=|youtu\.be\/)([^&\n?#]+)',
            r'youtube\.com\/embed\/([^&\n?#]+)',
            r'youtube\.com\/v\/([^&\n?#]+)',
            r'youtube\.com\/shorts\/([^&\n?#]+)'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None
    
    def extract_youtube_transcript(self, url: str) -> Dict[str, Union[str, bool]]:
        """YouTube 자막 추출 (최신 API 사용)"""
        try:
            video_id = self.extract_youtube_video_id(url)
            if not video_id:
                return {'success': False, 'error': '유효하지 않은 YouTube URL입니다.'}
            
            print(f"🔍 비디오 ID: {video_id}")
            
            # 최신 API 사용
            api = YouTubeTranscriptApi()
            
            # 다양한 언어로 시도
            languages_to_try = [['ko'], ['en'], ['a.ko'], ['a.en']]
            
            for languages in languages_to_try:
                try:
                    print(f"🔄 {languages[0]} 언어 자막 시도 중...")
                    fetched_transcript = api.fetch(video_id, languages=languages)
                    
                    # FetchedTranscriptSnippet 객체의 속성 접근
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    print(f"✅ 자막 추출 성공! (언어: {languages[0]}, 길이: {len(content)}자)")
                    return {
                        'content': content,
                        'success': True,
                        'language': languages[0],
                        'video_id': video_id,
                        'length': len(content)
                    }
                except Exception as e:
                    print(f"❌ {languages[0]} 실패: {str(e)}")
                    continue
            
            # 사용 가능한 자막 목록 확인
            try:
                print("📋 사용 가능한 자막 목록 확인 중...")
                transcript_list = api.list(video_id)
                
                if len(transcript_list) > 0:
                    first_transcript = transcript_list[0]
                    print(f"🔄 첫 번째 자막 언어로 시도: {first_transcript.language_code}")
                    
                    fetched_transcript = first_transcript.fetch()
                    content = ' '.join([item.text for item in fetched_transcript])
                    content = re.sub(r'\s+', ' ', content).strip()
                    
                    print(f"✅ 자막 추출 성공! (언어: {first_transcript.language_code}, 길이: {len(content)}자)")
                    return {
                        'content': content,
                        'success': True,
                        'language': first_transcript.language_code,
                        'video_id': video_id,
                        'length': len(content)
                    }
                
            except Exception as e:
                print(f"❌ 자막 목록 확인 실패: {str(e)}")
            
            return {'success': False, 'error': '자막을 찾을 수 없거나 네트워크 제한이 있습니다.'}
            
        except Exception as e:
            return {'success': False, 'error': f'오류 발생: {str(e)}'}
    
    async def summarize_with_gemini(self, content: str, custom_prompt: str = '') -> Dict[str, Union[str, bool]]:
        """
        Gemini로 콘텐츠 요약
        
        Args:
            content: 요약할 텍스트
            custom_prompt: 사용자 정의 프롬프트
            
        Returns:
            Dict: {'summary': str, 'success': bool, 'error': str}
        """
        try:
            # 기본 요약 프롬프트
            default_prompt = """
다음은 유튜브 영상의 자막입니다. 이를 학습자를 위해 체계적으로 요약해주세요.

## 📝 요약 구조
1. **핵심 주제와 목표** - 이 영상이 다루는 주요 내용
2. **주요 개념과 내용** - 핵심 포인트를 3-5개로 정리
3. **실습/예제 내용** - 코드나 실습이 있다면 상세히 기록
4. **학습 포인트와 결론** - 중요한 takeaway

각 섹션을 명확하게 구분하고, 마크다운 형식으로 구조화해서 작성해주세요.
"""
            
            # 프롬프트 선택
            system_prompt = custom_prompt if custom_prompt else default_prompt
            
            # 콘텐츠가 너무 긴 경우 축약 (Gemini 토큰 제한 고려)
            max_content_length = 30000
            if len(content) > max_content_length:
                content = content[:max_content_length] + "\n\n... (내용이 길어 일부만 요약됩니다)"
                print(f"⚠️ 콘텐츠가 길어서 {max_content_length}자로 축약됨")
            
            # 전체 프롬프트 구성
            full_prompt = f"{system_prompt}\n\n## 원본 콘텐츠:\n{content}"
            
            print(f"🧠 Gemini로 요약 생성 중... (콘텐츠 길이: {len(content)}자)")
            
            # Gemini 호출
            response = await asyncio.get_event_loop().run_in_executor(
                None, 
                lambda: self.model.generate_content(
                    full_prompt,
                    generation_config={
                    'temperature': 0.3,
                    'max_output_tokens': 2048,
                    'top_p': 0.8,
                    'top_k': 40
                }
                )
            )
            
            if not response.text:
                return {
                    'summary': '',
                    'success': False,
                    'error': 'Gemini에서 응답을 생성하지 못했습니다.'
                }
            
            summary = response.text.strip()
            
            print(f"✅ Gemini 요약 생성 성공! (길이: {len(summary)}자)")
            return {
                'summary': summary,
                'success': True,
                'error': None,
                'tokens_used': len(full_prompt.split()) + len(summary.split())
            }
            
        except Exception as e:
            print(f"❌ Gemini 요약 생성 오류: {str(e)}")
            return {
                'summary': '',
                'success': False,
                'error': f'요약 생성 중 오류가 발생했습니다: {str(e)}'
            }
    
    def create_notion_page_with_summary(self, title: str, summary: str, original_content: str, 
                                       url: str, metadata: dict = None) -> Dict[str, Union[str, bool]]:
        """
        Notion에 요약된 페이지 생성 (기존 방식 사용)
        
        Args:
            title: 페이지 제목
            summary: Gemini 요약 내용
            original_content: 원본 자막 내용
            url: YouTube URL
            metadata: 추가 메타데이터
            
        Returns:
            Dict: {'success': bool, 'page_url': str, 'error': str}
        """
        try:
            # Notion API 헤더
            headers = {
                "Authorization": f"Bearer {self.notion_token}",
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
            
            # 현재 시간
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 제목이 없으면 자동 생성
            if not title:
                title = f"YouTube 요약 - {current_time}"
            
            # 새 페이지 생성
            new_page = {
                "parent": {"page_id": self.parent_page_id},
                "properties": {
                    "title": [{"text": {"content": title}}]
                }
            }
            
            # 페이지 생성 요청
            page_response = requests.post(
                "https://api.notion.com/v1/pages", 
                headers=headers, 
                data=json.dumps(new_page)
            )
            
            if page_response.status_code != 200:
                return {
                    'success': False,
                    'error': f'페이지 생성 실패: {page_response.text}'
                }
            
            page_data = page_response.json()
            page_id = page_data['id']
            page_url = page_data['url']
            
            print(f"✅ Notion 페이지 생성 완료: {page_id}")
            
            # 페이지 콘텐츠 추가
            blocks = []
            
            # 1. YouTube URL 링크
            blocks.append({
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {"type": "text", "text": {"content": "🔗 원본 영상: "}},
                        {"type": "text", "text": {"content": url, "link": {"url": url}}}
                    ]
                }
            })
            
            # 2. 메타데이터
            if metadata:
                blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [
                            {"type": "text", "text": {"content": f"📊 언어: {metadata.get('language', 'unknown')} | 길이: {metadata.get('length', 0):,}자 | 생성일: {current_time}"}}
                        ]
                    }
                })
            
            # 3. 구분선
            blocks.append({"object": "block", "type": "divider", "divider": {}})
            
            # 4. AI 요약 제목
            blocks.append({
                "object": "block",
                "type": "heading_1",
                "heading_1": {
                    "rich_text": [{"type": "text", "text": {"content": "🤖 Gemini AI 요약"}}]
                }
            })
            
            # 5. 요약 내용 (2000자씩 나누어 추가)
            summary_chunks = [summary[i:i+2000] for i in range(0, len(summary), 2000)]
            for chunk in summary_chunks:
                blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": chunk}}]
                    }
                })
            
            # 6. 구분선
            blocks.append({"object": "block", "type": "divider", "divider": {}})
            
            # 7. 원문 자막 (토글 블록으로)
            original_chunks = [original_content[i:i+2000] for i in range(0, len(original_content), 2000)]
            original_blocks = []
            
            for chunk in original_chunks:
                original_blocks.append({
                    "object": "block",
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [{"type": "text", "text": {"content": chunk}}]
                    }
                })
            
            blocks.append({
                "object": "block",
                "type": "toggle",
                "toggle": {
                    "rich_text": [{"type": "text", "text": {"content": "📄 원문 자막 보기"}}],
                    "children": original_blocks[:50]  # 최대 50개 블록
                }
            })
            
            # 페이지에 블록 추가
            blocks_response = requests.patch(
                f"https://api.notion.com/v1/blocks/{page_id}/children",
                headers=headers,
                data=json.dumps({"children": blocks})
            )
            
            if blocks_response.status_code != 200:
                print(f"⚠️ 콘텐츠 추가 부분 실패: {blocks_response.text}")
            else:
                print("✅ Notion 페이지 콘텐츠 추가 완료!")
            
            return {
                'success': True,
                'page_url': page_url,
                'page_id': page_id,
                'error': None
            }
            
        except Exception as e:
            print(f"❌ Notion 페이지 생성 오류: {str(e)}")
            return {
                'success': False,
                'error': f'Notion 페이지 생성 중 오류: {str(e)}'
            }
    
    async def process_youtube_to_notion(self, url: str, custom_prompt: str = '', 
                                      custom_title: str = '') -> Dict[str, any]:
        """
        YouTube URL을 처리하여 Notion에 요약 페이지 생성 (완전한 파이프라인)
        
        Args:
            url: YouTube URL
            custom_prompt: 사용자 정의 요약 프롬프트
            custom_title: 사용자 정의 페이지 제목
            
        Returns:
            Dict: 전체 처리 결과
        """
        result = {
            'url': url,
            'transcript_result': {},
            'summary_result': {},
            'notion_result': {},
            'overall_success': False,
            'errors': []
        }
        
        try:
            print(f"🚀 YouTube → Gemini → Notion 파이프라인 시작!")
            print(f"📺 URL: {url}")
            
            # 1. 자막 추출
            print("\n📺 1단계: YouTube 자막 추출...")
            transcript_result = self.extract_youtube_transcript(url)
            result['transcript_result'] = transcript_result
            
            if not transcript_result['success']:
                result['errors'].append(transcript_result['error'])
                print(f"❌ 자막 추출 실패: {transcript_result['error']}")
                return result
            
            # 2. Gemini 요약 생성
            print("\n🧠 2단계: Gemini AI 요약 생성...")
            summary_result = await self.summarize_with_gemini(
                transcript_result['content'], 
                custom_prompt
            )
            result['summary_result'] = summary_result
            
            if not summary_result['success']:
                result['errors'].append(summary_result['error'])
                print(f"❌ 요약 생성 실패: {summary_result['error']}")
                return result
            
            # 3. Notion 페이지 생성
            print("\n📝 3단계: Notion 페이지 생성...")
            
            # 제목 설정
            title = custom_title if custom_title else f"YouTube 요약: {transcript_result.get('video_id', 'Unknown')}"
            
            # 메타데이터 준비
            metadata = {
                'language': transcript_result.get('language', 'unknown'),
                'length': transcript_result.get('length', 0),
                'video_id': transcript_result.get('video_id', ''),
                'tokens_used': summary_result.get('tokens_used', 0)
            }
            
            notion_result = self.create_notion_page_with_summary(
                title=title,
                summary=summary_result['summary'],
                original_content=transcript_result['content'],
                url=url,
                metadata=metadata
            )
            result['notion_result'] = notion_result
            
            if not notion_result['success']:
                result['errors'].append(notion_result['error'])
                print(f"❌ Notion 페이지 생성 실패: {notion_result['error']}")
                return result
            
            # 4. 성공!
            result['overall_success'] = True
            print("\n🎉 전체 파이프라인 완료!")
            print(f"📄 생성된 Notion 페이지: {notion_result['page_url']}")
            
        except Exception as e:
            error_msg = f"파이프라인 처리 중 오류: {str(e)}"
            result['errors'].append(error_msg)
            print(f"❌ {error_msg}")
        
        return result

print("📺 YouTubeGeminiNotionAgent 클래스 정의 완료!")

## 🚀 통합 Agent 초기화

In [ ]:
# Agent 초기화
if all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    agent = YouTubeGeminiNotionAgent(
        notion_token=NOTION_TOKEN,
        parent_page_id=PARENT_PAGE_ID,
        google_ai_api_key=GEMINI_API_KEY,
        database_id=NOTION_DATABASE_ID  # 선택사항
    )
    print("✅ 통합 Agent 초기화 성공!")
else:
    print("❌ 필요한 API 키가 설정되지 않았습니다!")
    print("💡 위의 환경 설정 셀을 먼저 확인해주세요.")

## 🎯 완전 자동화 실행

In [ ]:
# YouTube URL 입력
print("🔗 요약하고 Notion에 저장할 YouTube URL을 입력하세요:")
youtube_url = input("URL: ").strip()

if not youtube_url:
    youtube_url = "https://youtu.be/6hSjOCUmjGQ?si=ES5FOYHaFjWtguUl"  # 기본 테스트 URL
    print(f"📺 기본 URL 사용: {youtube_url}")

print(f"📺 처리할 URL: {youtube_url}")

In [ ]:
# 사용자 정의 옵션 설정
print("\n💬 사용자 정의 요약 프롬프트 (엔터 시 기본값):")
custom_prompt = input().strip()

print("\n📝 사용자 정의 페이지 제목 (엔터 시 자동 생성):")
custom_title = input().strip()

print(f"\n📋 설정 완료:")
print(f"  • 요약 프롬프트: {'사용자 정의' if custom_prompt else '기본값'}")
print(f"  • 페이지 제목: {'사용자 정의' if custom_title else '자동 생성'}")

In [ ]:
# 🚀 완전 자동화 실행!
if all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    
    print("\n" + "🚀 YouTube → Gemini → Notion 완전 자동화 시작!".center(80, "="))
    
    # 비동기 실행
    result = await agent.process_youtube_to_notion(
        url=youtube_url,
        custom_prompt=custom_prompt,
        custom_title=custom_title
    )
    
    # 📊 최종 결과 출력
    print("\n" + "📊 최종 결과".center(80, "="))
    
    if result['overall_success']:
        print("✅ 모든 단계 성공적으로 완료!")
        
        # 자막 정보
        transcript = result['transcript_result']
        print(f"\n📺 자막 추출 결과:")
        print(f"  • 언어: {transcript.get('language', 'unknown')}")
        print(f"  • 길이: {transcript.get('length', 0):,}자")
        print(f"  • 비디오 ID: {transcript.get('video_id', 'unknown')}")
        
        # 요약 정보
        summary_result = result['summary_result']
        print(f"\n🧠 Gemini 요약 결과:")
        print(f"  • 요약 길이: {len(summary_result.get('summary', ''))}자")
        print(f"  • 토큰 사용량: {summary_result.get('tokens_used', 0):,}")
        
        # Notion 페이지 정보
        notion_result = result['notion_result']
        print(f"\n📝 Notion 페이지 생성 결과:")
        print(f"  • 페이지 ID: {notion_result.get('page_id', 'unknown')}")
        
        # 🎯 생성된 Notion 페이지 링크
        print(f"\n" + "🎯 생성된 Notion 페이지".center(80, "="))
        print(f"📄 {notion_result.get('page_url', '')}")
        print("="*80)
        
        # 요약 내용 미리보기
        summary_preview = summary_result.get('summary', '')[:300] + "..." if len(summary_result.get('summary', '')) > 300 else summary_result.get('summary', '')
        print(f"\n📋 Gemini 요약 미리보기:")
        print(f"{summary_preview}")
        
    else:
        print("❌ 처리 실패:")
        for error in result['errors']:
            print(f"  • {error}")
        
        if any('network' in error.lower() or 'proxy' in error.lower() for error in result['errors']):
            print("\n🌐 현재 환경의 네트워크 제한으로 인한 문제일 수 있습니다.")
            print("💡 로컬 환경에서는 완벽하게 작동할 것입니다!")
        
else:
    print("❌ 필요한 API 키가 설정되지 않았습니다!")
    print("💡 환경 설정을 먼저 완료해주세요.")

## 🔄 추가 URL 처리

In [ ]:
# 추가 URL 처리
print("🔗 추가로 처리할 YouTube URL이 있습니까? (y/n)")
continue_processing = input().strip().lower()

if continue_processing == 'y' and all([NOTION_TOKEN, PARENT_PAGE_ID, GEMINI_API_KEY]):
    another_url = input("추가 YouTube URL: ").strip()
    another_prompt = input("요약 스타일 (엔터 시 기본값): ").strip()
    another_title = input("페이지 제목 (엔터 시 자동 생성): ").strip()
    
    if another_url:
        print(f"\n🚀 추가 처리 시작: {another_url}")
        
        # 추가 실행
        additional_result = await agent.process_youtube_to_notion(
            url=another_url,
            custom_prompt=another_prompt,
            custom_title=another_title
        )
        
        if additional_result['overall_success']:
            print("\n✅ 추가 처리 성공!")
            print(f"📄 추가 Notion 페이지: {additional_result['notion_result'].get('page_url', '')}")
        else:
            print(f"\n❌ 추가 처리 실패: {additional_result['errors']}")
else:
    print("처리를 종료합니다.")

## 📋 완료!

### 🎉 YouTube + Gemini + Notion 통합 시스템 완성!

**🔄 완전한 자동화 파이프라인:**
1. ✅ **YouTube 자막 추출** - 최신 API로 다양한 언어 지원
2. ✅ **Gemini AI 요약** - Google의 최신 AI로 고품질 요약
3. ✅ **Notion 페이지 생성** - 구조화된 페이지로 자동 저장
4. ✅ **사용자 정의** - 프롬프트와 제목 커스터마이징 지원

### 💡 주요 특징:
- 🎯 **원클릭 자동화**: URL 입력만으로 전체 프로세스 완료
- 📊 **풍부한 메타데이터**: 언어, 길이, 토큰 사용량 등 상세 정보
- 📝 **구조화된 페이지**: 요약 + 원문 자막을 깔끔하게 정리
- 🔗 **편리한 접근**: 생성된 Notion 페이지 직접 링크 제공

### 🛠️ 환경 설정 요구사항:
```env
NOTION_TOKEN=" "
notion_parent_page=" "
GEMINI_API_KEY=" "
NOTION_DATABASE_ID=" "
```

### 🔗 관련 파일:
- **[youtube_gemini_summary.py](computer:///mnt/user-data/outputs/youtube_gemini_summary.py)** - 단독 스크립트 버전
- **[ai_agent_backend.py](computer:///mnt/user-data/outputs/ai_agent_backend.py)** - 전체 백엔드 시스템

**이제 YouTube 영상을 입력하면 자동으로 요약되어 Notion에 깔끔하게 정리됩니다!** 🚀✨